Day 11 (https://adventofcode.com/2023/day/11)

In [1]:
import numpy as np
import itertools

with open('./inputs/day11.txt', 'r') as f:
    orig_galaxy_map = np.array([list(l.strip()) for l in f.readlines()])
    galaxy_map = orig_galaxy_map.copy()

    vertical_expansions = []
    for i in range(galaxy_map.shape[0]-1, -1, -1):
        if np.sum(galaxy_map[i, :]=='#')==0:
            vertical_expansions.append(i)
            galaxy_map = np.insert(galaxy_map, i, galaxy_map[i, :].copy(), axis=0)

    horizontal_expansions = []
    for i in range(galaxy_map.shape[1]-1, -1, -1):
        if np.sum(galaxy_map[:, i]=='#')==0:
            horizontal_expansions.append(i)
            galaxy_map = np.insert(galaxy_map, i, galaxy_map[:, i].copy(), axis=1)

    # sum manhattan distances for answer
    part1_ans = sum(abs(i[0]-j[0]) + abs(i[1]-j[1]) for i, j in itertools.combinations(np.argwhere(galaxy_map=='#'), 2))
    print('Answer to Day 11, Part 1:', part1_ans)

    # custom function for part 2
    def galaxy_dist(galaxy1, galaxy2):
        output = abs(galaxy1[0]-galaxy2[0]) + abs(galaxy1[1]-galaxy2[1])
        output += 999999*len([i for i in vertical_expansions if i < max(galaxy1[0], galaxy2[0]) and i > min(galaxy1[0], galaxy2[0])])
        output += 999999*len([i for i in horizontal_expansions if i < max(galaxy1[1], galaxy2[1]) and i > min(galaxy1[1], galaxy2[1])])
        return output

    part2_ans = sum(galaxy_dist(i, j) for i, j in itertools.combinations(np.argwhere(orig_galaxy_map=='#'), 2))
    print('Answer to Day 11, Part 2:', part2_ans)

Answer to Day 11, Part 1: 10885634
Answer to Day 11, Part 2: 707505470642


Day 12 (https://adventofcode.com/2023/day/12)

In [2]:
import itertools
import re

with open('./inputs/day12.txt', 'r') as f:
    report = [
        {
            'report_str': l.split()[0],
            'report_list': [int(x) for x in l.split()[1].split(',')]
        }
        for l in f.readlines()
    ]

    # function to check whether given string-list configuration is valid
    def is_valid(report_str, report_list):
        runs = list(filter(lambda x: len(x) > 0, report_str.split('.')))
        return all((len(x)==n for x, n in zip(runs, report_list))) and len(runs) == len(report_list)
    
    # function to iterate through different possibilities of dirty string
    def possibilities(dirty_report_str):
        for i in itertools.product('.#', repeat=dirty_report_str.count('?')):
            stuff = iter(i)
            yield ''.join((next(stuff) if c=='?' else c for c in dirty_report_str))

    # just brute-force checking validity for part 1
    part1_ans = sum([sum(is_valid(p, d['report_list']) for p in possibilities(d['report_str'])) for d in report])
    print('Answer to Day 12, Part 1:', part1_ans)

    # ugh: can't brute force part 2 :-(
    report2 = [
        {
            'report_str': '?'.join([d['report_str']]*5),
            'report_list': sum([d['report_list']]*5, start=[])
        }
        for d in report
    ]

    cached_results = {}
    def get_num_solutions(s, l):
        if len(l)==0:
            return int('#' not in s)

        cached_result = cached_results.get((s.strip('.'), tuple(l)))
        if cached_result is not None:
            return cached_result
        
        num_solutions = 0
        for m in re.finditer(rf'(?=((#|\?){{{l[0]}}})(\.|\?|$))', s):
            if len(s[m.end(1):]) < sum(l[1:]) + len(l[1:]) or '#' in s[:m.start(1)]:
                break

            num_solutions += get_num_solutions(s[m.end(1)+1:], l[1:])

        cached_results[(s.strip('.'), tuple(l))] = num_solutions
        return num_solutions

    part2_ans = sum([get_num_solutions(d['report_str'], d['report_list']) for d in report2])
    print('Answer to Day 12, Part 2:', part2_ans)

Answer to Day 12, Part 1: 7857
Answer to Day 12, Part 2: 28606137449920
